# iQuHack 2026 - Quantum Entanglement Distillation Game

In this game, you design quantum circuits to distill noisy Bell pairs and claim edges in a network.

**Game Flow**: Register -> Select Starting Node -> Design Circuits -> Claim Edges -> Score Points

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=26572fcb5e7e91e2fc454f0aa5595d1eac9e15e31375e4a46c3cd67d35c439b5
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee

In [1]:
from client import GameClient
from visualization import GraphTool
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import json
from pathlib import Path

## Session Management

Save/load your session to avoid re-registering.

In [2]:
SESSION_FILE = Path("session.json")

def save_session(client):
    if client.api_token:
        with open(SESSION_FILE, "w") as f:
            json.dump({"api_token": client.api_token, "player_id": client.player_id, "name": client.name}, f)
        print(f"Session saved.")

def load_session():
    if not SESSION_FILE.exists():
        return None
    with open(SESSION_FILE) as f:
        data = json.load(f)
    client = GameClient(api_token=data.get("api_token"))
    client.player_id = data.get("player_id")
    client.name = data.get("name")
    status = client.get_status()
    if status:
        print(f"Resumed: {client.player_id} | Score: {status.get('score', 0)} | Budget: {status.get('budget', 0)}")
        return client
    return None

In [4]:
# Try to resume existing session
client = load_session()

if not client:
    print("No saved session. Register below.")

No saved session. Register below.


## Step 1: Register

Skip this if you resumed a session above.

In [5]:
if client and client.api_token:
    print(f"Already registered as {client.player_id}")
else:
    client = GameClient()

    # CHANGE THESE to your unique values
    PLAYER_ID = "GiselleRocio"
    PLAYER_NAME = "GiselleRocio"

    result = client.register(PLAYER_ID, PLAYER_NAME, location="remote")

    if result.get("ok"):
        print(f"Registered! Token: {client.api_token[:20]}...")
        candidates = result["data"].get("starting_candidates", [])
        print(f"\nStarting candidates ({len(candidates)}):")
        for c in candidates:
            print(f"  - {c['node_id']}: {c['utility_qubits']} qubits, +{c['bonus_bell_pairs']} bonus")
        save_session(client)
    else:
        print(f"Failed: {result.get('error', {}).get('message')}")

Registered! Token: Wp0jeQrP09OiQxWtCWnD...

Starting candidates (4):
  - Groningen, Netherlands: 2 qubits, +0 bonus
  - Leiden, Netherlands: 1 qubits, +1 bonus
  - Munich, Germany: 3 qubits, +2 bonus
  - Surabaya, Indonesia: 4 qubits, +0 bonus
Session saved.


## Step 2: Select Starting Node

In [160]:
status = client.get_status()

if status.get('starting_node'):
    print(f"Starting node: {status['starting_node']}")
    print(f"Budget: {status['budget']} | Score: {status['score']}")
else:
    print("Select a starting node from the candidates shown above.")
    # Uncomment and modify:
    result = client.select_starting_node("Munich, Germany")
    print(result)

Select a starting node from the candidates shown above.
{'ok': True, 'data': {'success': True, 'starting_node': 'Munich, Germany', 'score': 44, 'budget': 40}}


## Step 3: Explore the Network

In [7]:
client.print_status()

Player: GiselleRocio (GiselleRocio)
Score: 0 | Budget: 40 bell pairs
Active: Yes
Starting node: Munich, Germany
Owned: 1 nodes, 0 edges
Claimable edges: 4
  - ['Innsbruck, Austria', 'Munich, Germany']: threshold=0.90, difficulty=1
  - ['Munich, Germany', 'Salzburg, Austria']: threshold=0.90, difficulty=1
  - ['Munich, Germany', 'Nuremberg, Germany']: threshold=0.90, difficulty=1
  ... and 1 more


In [ ]:
# Visualize the network (focused around your nodes)
viz = GraphTool(client.get_cached_graph())
owned = set(client.get_status().get('owned_nodes', []))
viz.render(owned)

## Step 4: Design a Distillation Circuit

For N Bell pairs, qubits are paired outside-in:
- Pair 0: qubits {0, 2N-1}
- Pair 1: qubits {1, 2N-2}
- ...
- Final pair: qubits {N-1, N}

Design a circuit that improves fidelity through LOCC operations.

In [143]:
import requests

def safe_claim_edge_with_qasm(client, edge_id, qasm_str, flag_bit: int, num_bell_pairs: int):
    payload = {
        "player_id": client.player_id,
        "edge": [edge_id[0], edge_id[1]],
        "num_bell_pairs": int(num_bell_pairs),
        "circuit_qasm": qasm_str,
        "flag_bit": int(flag_bit),
    }
    try:
        return client._post("/v1/claim_edge", payload)
    except requests.HTTPError as e:
        print("\n=== HTTPError ===")
        print("status:", e.response.status_code)
        try:
            print("response json:", e.response.json())
        except Exception:
            print("response text:", e.response.text[:2000])
        print("payload keys:", list(payload.keys()))
        print("edge:", payload["edge"], "N:", payload["num_bell_pairs"], "flag_bit:", payload["flag_bit"])
        raise



In [136]:
def qasm_bbpssw_N2():
    # bits: c[0]=a_meas, c[1]=b_meas, c[2]=flag
    # flag_bit = 2
    return """OPENQASM 3.0;
include "stdgates.inc";

qubit[4] q;
bit[3] c;

// Pairing for N=2:
// ancilla pair: (q0, q3)
// data/output pair: (q1, q2)

// Bilateral CNOT: data -> ancilla (local on each side)
cx q[1], q[0];   // Alice local
cx q[2], q[3];   // Bob local

// Measure ancilla qubits
measure q[0] -> c[0];
measure q[3] -> c[1];

// Keep iff equal
c[2] = c[0] ^ c[1];
""", 2


In [137]:
def qasm_dejmps_lite_N2():
    return """OPENQASM 3.0;
include "stdgates.inc";

qubit[4] q;
bit[3] c;

// Local basis tweaks (DEJMPS-ish):
// Apply H on the ancilla qubits to mix phase/bit information a bit.
// (Still LOCC: only 1-qubit gates.)
h q[0];
h q[3];

// Bilateral CNOT: data -> ancilla (local on each side)
cx q[1], q[0];
cx q[2], q[3];

// Measure ancilla
measure q[0] -> c[0];
measure q[3] -> c[1];

// Keep iff equal
c[2] = c[0] ^ c[1];
""", 2


In [138]:
def qasm_dejmps_lite_N2():
    return """OPENQASM 3.0;
include "stdgates.inc";

qubit[4] q;
bit[3] c;

// Local basis tweaks (DEJMPS-ish):
// Apply H on the ancilla qubits to mix phase/bit information a bit.
// (Still LOCC: only 1-qubit gates.)
h q[0];
h q[3];

// Bilateral CNOT: data -> ancilla (local on each side)
cx q[1], q[0];
cx q[2], q[3];

// Measure ancilla
measure q[0] -> c[0];
measure q[3] -> c[1];

// Keep iff equal
c[2] = c[0] ^ c[1];
""", 2


In [144]:
def try_to_cross_threshold(client, edge_id, threshold: float = 0.9, tries_each: int = 10, verbose: bool = True):
    candidates = [
        ("BBPSSW_N2", qasm_bbpssw_N2),
        ("DEJMPS_lite_N2", qasm_dejmps_lite_N2),
    ]

    best = None

    for name, builder in candidates:
        qasm, flag_bit = builder()

        for k in range(tries_each):
            resp = safe_claim_edge_with_qasm(
                client,
                edge_id=edge_id,
                qasm_str=qasm,
                flag_bit=flag_bit,
                num_bell_pairs=2
            )

            data = resp.get("data", resp)

            F = float(data.get("fidelity", 0.0))
            p = float(data.get("success_probability", 0.0))
            thr = float(data.get("threshold", threshold))

            # Estos dos son los que el server realmente usa (según lo que ya viste)
            success = bool(data.get("success", False))
            is_valid = bool(data.get("is_valid", True))  # "válido" en el sentido de request/circuit

            if verbose:
                print(f"[{name} #{k+1}] success={success} is_valid={is_valid} F={F:.4f} p_succ={p:.4f} thr={thr}")

            # Guardar mejor resultado aunque el server no lo marque "success"
            if best is None or F > best["fidelity"]:
                best = {"protocol": name, "fidelity": F, "p_succ": p, "success": success, "is_valid": is_valid, "resp": data}

            # Consideramos “hit” si el server dice success OR si F supera el threshold que reporta
            if success or (F >= thr):
                return {"hit": True, "protocol": name, "fidelity": F, "p_succ": p, "success": success, "is_valid": is_valid, "resp": data}

    return {"hit": False, "best": best}


In [150]:
claimable = client.get_claimable_edges()
claimable_sorted = sorted(claimable, key=lambda e: (e['difficulty_rating'], e['base_threshold']))
target = claimable_sorted[0]
edge_id = tuple(target["edge_id"])

result = try_to_cross_threshold(client, edge_id, threshold=0.9, tries_each=10, verbose=True)
print(result)

[BBPSSW_N2 #1] success=True is_valid=False F=0.9698 p_succ=0.7450 thr=0.9
{'hit': True, 'protocol': 'BBPSSW_N2', 'fidelity': 0.9697986577181208, 'p_succ': 0.7449999999999997, 'success': True, 'is_valid': False, 'resp': {'success': True, 'is_valid': False, 'fidelity': 0.9697986577181208, 'success_probability': 0.7449999999999997, 'threshold': 0.9, 'remaining_budget': 24, 'is_active': True, 'score': 13, 'reward_changes': []}}


In [154]:
import time
import random

def _edge_key(edge_id):
    """Normaliza el edge para usarlo como key (orden estable)."""
    a, b = edge_id
    return tuple(sorted((a, b)))

def pick_next_edge(client, conquered_edges, attempts_by_edge, max_attempts_per_edge=20):
    """
    Elige el próximo edge a intentar:
    - que sea claimable
    - que no esté conquistado
    - que no haya superado max_attempts_per_edge
    Priorización: difficulty asc, threshold asc (como venías haciendo),
    con un poquito de ruido para no clavarte siempre en el mismo.
    """
    claimable = client.get_claimable_edges()
    if not claimable:
        return None

    candidates = []
    for e in claimable:
        ek = _edge_key(tuple(e["edge_id"]))
        if ek in conquered_edges:
            continue
        if attempts_by_edge.get(ek, 0) >= max_attempts_per_edge:
            continue

        # score: lo tuyo + micro-aleatoriedad
        score = (e["difficulty_rating"], e["base_threshold"], random.random() * 0.001)
        candidates.append((score, e))

    if not candidates:
        return None

    candidates.sort(key=lambda x: x[0])
    return candidates[0][1]  # el edge dict

def auto_conquer(
    client,
    threshold_default=0.9,
    tries_each=10,
    max_edges_to_claim=50,
    max_attempts_per_edge=20,
    stop_budget_below=2,
    verbose=True,
    sleep_between_edges=0.0
):
    """
    Loop principal:
    - elige edge nuevo
    - corre try_to_cross_threshold
    - si hit => se agrega a conquered_edges
    - si no hit => suma intentos y pasa a otro
    """
    conquered_edges = set()
    attempts_by_edge = {}

    claimed_count = 0

    while True:
        status = client.get_status()
        if not status:
            print("No status (¿registrada?).")
            break

        budget = status.get("budget", 0)
        if budget < stop_budget_below:
            print(f"\n⚠️ Budget bajo ({budget}). Corto.")
            break

        # Sincronizá con el server: por si ya tenés edges conquistados
        for e in status.get("owned_edges", []):
            conquered_edges.add(_edge_key(tuple(e)))

        if claimed_count >= max_edges_to_claim:
            print(f"\n✓ Llegaste al máximo de edges a conquistar ({max_edges_to_claim}).")
            break

        target = pick_next_edge(
            client,
            conquered_edges=conquered_edges,
            attempts_by_edge=attempts_by_edge,
            max_attempts_per_edge=max_attempts_per_edge
        )

        if target is None:
            print("\n✓ No hay más edges “buenos” para intentar (o ya los quemaste a intentos).")
            break

        edge_id = tuple(target["edge_id"])
        ek = _edge_key(edge_id)

        # (Opcional) usar threshold propio del edge si querés
        thr = float(target.get("base_threshold", threshold_default))

        if verbose:
            print("\n" + "="*70)
            print(f"→ Intentando edge: {edge_id} | diff={target['difficulty_rating']} | thr={thr:.2f} | budget={budget}")
            print(f"  intentos previos sobre este edge: {attempts_by_edge.get(ek, 0)} / {max_attempts_per_edge}")

        # Tu función (la que ya tenés)
        result = try_to_cross_threshold(
            client,
            edge_id=edge_id,
            threshold=thr,
            tries_each=tries_each,
            verbose=verbose
        )

        attempts_by_edge[ek] = attempts_by_edge.get(ek, 0) + 1

        if result.get("hit"):
            conquered_edges.add(ek)
            claimed_count += 1
            if verbose:
                print(f"\n🏁 CONQUISTADO: {edge_id} | F={result['fidelity']:.4f} | p={result['p_succ']:.4f} | protocol={result['protocol']}")
        else:
            if verbose:
                best = result.get("best", {})
                if best:
                    print(f"\n✗ No alcanzó threshold. Mejor F visto={best.get('fidelity', 0):.4f} (protocol={best.get('protocol')})")

        if sleep_between_edges > 0:
            time.sleep(sleep_between_edges)

    return {
        "claimed_count": claimed_count,
        "conquered_edges": conquered_edges,
        "attempts_by_edge": attempts_by_edge
    }


In [ ]:
summary = auto_conquer(
    client,
    tries_each=10,
    max_attempts_per_edge=5,
    max_edges_to_claim=30,
    verbose=True
)
print("Edges conquistados:", summary["claimed_count"])



→ Intentando edge: ('Geneva, Switzerland', 'Lyon, France') | diff=1 | thr=0.90 | budget=40
  intentos previos sobre este edge: 0 / 5
[BBPSSW_N2 #1] success=True is_valid=False F=0.9698 p_succ=0.7450 thr=0.9

🏁 CONQUISTADO: ('Geneva, Switzerland', 'Lyon, France') | F=0.9698 | p=0.7450 | protocol=BBPSSW_N2

→ Intentando edge: ('Aachen, Germany', 'Eindhoven, Netherlands') | diff=1 | thr=0.90 | budget=38
  intentos previos sobre este edge: 0 / 5
[BBPSSW_N2 #1] success=True is_valid=False F=0.9698 p_succ=0.7450 thr=0.9

🏁 CONQUISTADO: ('Aachen, Germany', 'Eindhoven, Netherlands') | F=0.9698 | p=0.7450 | protocol=BBPSSW_N2

→ Intentando edge: ('Geneva, Switzerland', 'Grenoble, France') | diff=1 | thr=0.90 | budget=38
  intentos previos sobre este edge: 0 / 5
[BBPSSW_N2 #1] success=True is_valid=False F=0.9698 p_succ=0.7450 thr=0.9

🏁 CONQUISTADO: ('Geneva, Switzerland', 'Grenoble, France') | F=0.9698 | p=0.7450 | protocol=BBPSSW_N2

→ Intentando edge: ('Grenoble, France', 'Lyon, France') | 

## Step 5: Claim an Edge

## Step 6: Check Progress

In [158]:
client.print_status()

Player: GiselleRocio (GiselleRocio)
Score: 44 | Budget: 1 bell pairs
Active: Yes
Starting node: Munich, Germany
Owned: 19 nodes, 28 edges
Claimable edges: 14
  - ['Geneva, Switzerland', 'Lausanne, Switzerland']: threshold=0.90, difficulty=1
  - ['Bern, Switzerland', 'Geneva, Switzerland']: threshold=0.90, difficulty=1
  - ['Lausanne, Switzerland', 'Lyon, France']: threshold=0.90, difficulty=2
  ... and 11 more


In [148]:
# View leaderboard
leaderboard = client.get_leaderboard()["leaderboard"]
print("Leaderboard:")
for i, p in enumerate(leaderboard[:10]):
    print(f"{i+1}. {p.get('player_id', 'Unknown'):20} Score: {p.get('score', 0)}")

Leaderboard:
1. diyamagnetism28      Score: 93
2. bloch_distiller      Score: 89
3. bloch                Score: 87
4. bloch_single_run_v8  Score: 85
5. Ram_23356_new_version1 Score: 83
6. Munich_id            Score: 77
7. test_claude_1523b    Score: 70
8. orion_the_first      Score: 69
9. npqc_1               Score: 63
10. willignoresbonuses   Score: 62


## Tips

- **Failed attempts are free** - only successful claims cost bell pairs
- **More bell pairs** can improve fidelity but cost more budget
- **Vertex rewards** are competitive - top players by claim strength earn rewards
- **Budget management** is key - if budget reaches 0, you're eliminated

Good luck!

In [159]:
# Restart game (uncomment to use)
result = client.restart()
print(result)

{'ok': True, 'data': {'success': True, 'budget': 40, 'is_active': True, 'starting_candidates': [{'node_id': 'Groningen, Netherlands', 'utility_qubits': 2, 'bonus_bell_pairs': 0, 'capacity': 5, 'latitude': 53.2194, 'longitude': 6.5665}, {'node_id': 'Leiden, Netherlands', 'utility_qubits': 1, 'bonus_bell_pairs': 1, 'capacity': 5, 'latitude': 52.1601, 'longitude': 4.497}, {'node_id': 'Munich, Germany', 'utility_qubits': 3, 'bonus_bell_pairs': 2, 'capacity': 9, 'latitude': 48.1351, 'longitude': 11.582}, {'node_id': 'Surabaya, Indonesia', 'utility_qubits': 4, 'bonus_bell_pairs': 0, 'capacity': 6, 'latitude': -7.2575, 'longitude': 112.7521}]}}
